# CSE 158 HW3
## Christina Leung, A15468909

In [1]:
import gzip
import random
import string
from sklearn import linear_model
from collections import defaultdict
import scipy.optimize
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
import numpy

In [2]:
def readJSON(path):
  for l in gzip.open(path, 'rt'):
    d = eval(l)
    u = d['userID']
    try:
      g = d['gameID']
    except Exception as e:
      g = None
    yield u,g,d

pairs_train = []
pairs_valid = []
games = set([])

i = 0
for user,game,d in readJSON("train.json.gz"):
  if i < 165000:
    pairs_train.append([user, game, True])
  else:
    pairs_valid.append([user, game, True])
  i += 1
  games.add(game)
games = list(games)

In [3]:
# QUESTION 1
print(len(pairs_valid))
for pair in pairs_valid:
    if pair[2] is False:
        continue
    user = pair[0]
    while True:
        game = random.choice(games)
        if (user, game, True) not in pairs_valid and (user, game, True) not in pairs_train:
            if (user, game, False) not in pairs_valid:
                pairs_valid.append((user, game, False))
                break
print(len(pairs_valid))            

10000
20000


In [4]:
gameCount = defaultdict(int)
totalPlayed = 0

for pair in pairs_train:
    gameCount[pair[1]] += 1
    totalPlayed += 1

mostPopular = [(gameCount[x], x) for x in gameCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalPlayed/2: break

correct = 0
total = 0
falsepos = 0
falseneg = 0
for pair in pairs_valid:
  total += 1
  g = pair[1]
  ans = pair[2]
  guess = False
  if g in return1:
    guess = True
  if guess == ans:
    correct += 1
  if guess == True and ans == False:
    falsepos +=1
  if guess == False and ans == True:
    falseneg += 1
print(falsepos)
print(falseneg)
print('accuracy:')
print(correct/total)

1457
4979
accuracy:
0.6782


## QUESTION 1
Accuracy: 0.6791

In [5]:
# QUESTION 2
return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalPlayed/1.5: break

correct = 0
total = 0
falsepos = 0
falseneg = 0
for pair in pairs_valid:
  total += 1
  g = pair[1]
  ans = pair[2]
  guess = False
  if g in return1:
    guess = True
  if guess == ans:
    correct += 1
  if guess == True and ans == False:
    falsepos +=1
  if guess == False and ans == True:
    falseneg += 1
print(falsepos)
print(falseneg)
print('accuracy:')
print(correct/total)

2659
3340
accuracy:
0.70005


## QUESTION 2
Uses the top 66.67%, accuracy of 0.69745

In [6]:
# QUESTION 3
user2game = {}
game2user = {}
for pair in pairs_train:
    user = pair[0]
    game = pair[1]
    if game in game2user and user not in game2user[game]:
        game2user[game].add(user)
    else:
        game2user[game] = set([user])
    if user in user2game and game not in user2game[user]:
        user2game[user].add(game)
    else :
        user2game[user] = set([game])
def Jaccard(u, g):
    usergames = user2game[u]
    scores = []
    s1 = game2user[g]
    for x in usergames:
        s2 = game2user[x]
        numer = len(s1.intersection(s2))
        denom = len(s1.union(s2))
        scores.append(numer/denom)
    if (len(scores) > 0):
        return max(scores)
    return 0

In [7]:
correct = 0
total = 0
falsepos = 0
falseneg = 0
for pair in pairs_valid:
    real = pair[2]
    guess = Jaccard(pair[0], pair[1])
    if (guess > 0.031):
        guess = True
    else:
        guess = False
    if guess == real:
        correct += 1
    if guess == True and real == False:
        falsepos +=1
    if guess == False and real == True:
        falseneg += 1
print(falsepos)
print(falseneg)
print('accuracy:')
print(correct/len(pairs_valid))

3107
3478
accuracy:
0.67075


## Question 3
Selected threshold is 0.031 because it yielded the maximum accuracy in the trials. Going higher or lower than that number in increments of 0.001 did not improve the accuracy.<br>
Different Threshold Trials:<br>
Threshold: 0.029, Accuracy: 0.6669<br>
Threshold: 0.030, Accuracy: 0.66775<br>
Threshold: 0.031, Accuracy: 0.6692<br>
Threshold: 0.032, Accuracy: 0.6686<br>
Threshold: 0.033, Accuracy: 0.666<br>

In [8]:
# QUESTION 4
return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalPlayed/1.5: break
def JaccardPop(u, g):
    usergames = set([])
    if u in user2game:
        usergames = user2game[u]
    scores = []
    s1 = set([])
    if g in game2user:
        s1 = game2user[g]
    for x in usergames:
        s2 = set([])
        if x in game2user:
            s2 = game2user[x]
        numer = len(s1.intersection(s2))
        denom = len(s1.union(s2))
        scores.append(numer/denom)
    jac = 0
    if (len(scores) > 0):
        jac = max(scores)
    if g in return1:
        jac += 0.015
    else:
        jac -= 0.007
    if jac > 0.035:
        return True
    else:
        return False
correct = 0
total = 0
falsepos = 0
falseneg = 0
for pair in pairs_valid:
    total += 1
    real = pair[2]
    guess = JaccardPop(pair[0], pair[1])
    if guess == real:
        correct += 1
    if guess == True and real == False:
        falsepos +=1
    if guess == False and real == True:
        falseneg += 1
print(falsepos)
print(falseneg)
print('accuracy:')
print(correct/total)

2993
3020
accuracy:
0.69935


## QUESTION 4
Accuracy: 0.6992

In [9]:
# QUESTION 5
predictions = open("predictions_Played.txt", 'w')
for l in open("pairs_Played.txt"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,g = l.strip().split('-')
  if JaccardPop(u, g) == True:
    predictions.write(u + '-' + g + ",1\n")
  else:
    predictions.write(u + '-' + g + ",0\n")

predictions.close()

## Question 5
user name: christinaleung123<br>
Kaggle Accuracy: 0.71040

In [10]:
# QUESTION 6
def readCatJSON(path):
  for l in gzip.open(path, 'rt'):
    d = eval(l)
    r = d['text']
    try:
      g = d['genreID']
    except Exception as e:
      g = None
    yield r,g,d
    
catDict = {
  "Action": 0,
  "Strategy": 1,
  "RPG": 2,
  "Adventure": 3,
  "Sport": 4
}

cat_train = []
cat_valid = []
i = 0
for r,g,d in readCatJSON("train_Category.json.gz"):
    if i < 165000:
        cat_train.append((r, g))
    else:
        cat_valid.append((r, g))
    i += 1
print(len(cat_train))
print(len(cat_valid))

165000
10000


In [11]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in cat_train:
  r = ''.join([c for c in d[0].lower() if not c in punctuation])
  for w in r.split():
    wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words2 = [x for x in counts[:10]]
print(words2)
words = [x[1] for x in counts[:1000]]

[(544597, 'the'), (317620, 'and'), (305414, 'a'), (291882, 'to'), (245359, 'game'), (227234, 'of'), (208417, 'is'), (200633, 'you'), (195953, 'i'), (190966, 'it')]


## Question 6
format is (count, word)<br>
[(544597, 'the'), (317620, 'and'), (305414, 'a'), (291882, 'to'), (245359, 'game'), (227234, 'of'), (208417, 'is'), (200633, 'you'), (195953, 'i'), (190966, 'it')]

In [12]:
# QUESTION 7
words = [x[1] for x in counts[:1000]]
wordId = dict(zip(words, range(len(words))))
def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum.lower() if not c in punctuation])
    for w in r.split():
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat
X = [feature(d[0]) for d in cat_train]
y = [d[1] for d in cat_train]
mod = linear_model.LogisticRegression(C=1.0, class_weight='balanced', max_iter=10000) # Fit classifier
mod.fit(X, y)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=10000, multi_class='auto', n_jobs=None,
                   penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
                   verbose=0, warm_start=False)

In [13]:
X_val = [feature(d[0]) for d in cat_valid]
y_val = [d[1] for d in cat_valid]
pred = mod.predict(X_val)
TP_ = numpy.logical_and(pred, y_val) # Calculate accuracy
TN_ = numpy.logical_and(numpy.logical_not(pred), numpy.logical_not(y_val))
TP = sum(TP_)
TN = sum(TN_)
print('accuracy:')
print((TP + TN) / len(cat_valid))

accuracy:
0.6271


## Question 7
Accuracy: 0.6269

In [14]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
stemmer = PorterStemmer()
stops = stopwords.words('english')
for d in cat_train:
  r = ''.join([c for c in d[0].lower() if not c in punctuation])
  for w in r.split():
    w = stemmer.stem(w)
    if w not in stops:
        wordCount[w] += 1
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse() 

In [15]:
# QUESTION 8  
words = [x[1] for x in counts[:5000]]
wordId = dict(zip(words, range(len(words))))
def feature2(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum.lower() if not c in punctuation])
    for w in r.split():
        w = stemmer.stem(w)
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat
X = [feature2(d[0]) for d in cat_train]
X = scipy.sparse.lil_matrix(X)
y = [d[1] for d in cat_train]
mod = linear_model.LogisticRegression(C=1.0E-3, class_weight='balanced', max_iter = 10000) # Fit classifier
#mod = linear_model.Ridge(1.0E5, fit_intercept=False)
mod.fit(X, y)

LogisticRegression(C=0.001, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=10000, multi_class='auto', n_jobs=None,
                   penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
                   verbose=0, warm_start=False)

In [16]:
X_val = [feature2(d[0]) for d in cat_valid]
X_val = scipy.sparse.lil_matrix(X_val)
y_val = [d[1] for d in cat_valid]
pred = mod.predict(X_val)
TP_ = numpy.logical_and(pred, y_val) # Calculate accuracy
TN_ = numpy.logical_and(numpy.logical_not(pred), numpy.logical_not(y_val))
TP = sum(TP_)
TN = sum(TN_)
print((TP + TN) / len(cat_valid))

0.6833


In [17]:
def readJSON2(path):
  for l in gzip.open(path, 'rt'):
    d = eval(l)
    u = d['userID']
    try:
      g = d['gameID']
    except Exception as e:
      g = None
    yield u,g,d
X_test = []
for u,_,d in readJSON2("test_Category.json.gz"):
    X_test.append(d['text'])
X_test = [feature2(d) for d in X_test]
pred = mod.predict(X_test)
i = 0
predictions = open("predictions_Category.txt", 'w')
predictions.write("userID-reviewID,prediction\n")
for u,_,d in readJSON2("test_Category.json.gz"):
  cat = pred[i]
  i += 1
  predictions.write(u + '-' + d['reviewID'] + "," + str(cat) + "\n")
predictions.close()

## QUESTION 8
Used stemming and removed stopwords<Br>
    
Using dictionary size of 500:<br>
C = 1.0, accuracy = 0.6076<br>
C = 1.0E-5, accuracy = 0.5092<br>
C = 1.0E-4, accuracy = 0.5735<br>
C = 1.0E-3, accuracy = 0.606<br>

Using dictionary size of 1000:<br>
C = 1.0, accuracy = 0.6458<br>
C = 1.0E-5, accuracy = 0.5164<br>
C = 1.0E-4, accuracy = 0.5946<br>
C = 1.0E-3, accuracy = 0.6336<br>

Using dictionary size of 5000:<br>
C = 1.0, accuracy = 0.7286<br>
C = 1.0E-3, accuracy = 0.6833<br>
C = 1.0E-2, accuracy = 0.7257<br>
C = 1.0E-1, accuracy = 0.7293<br>

Best performance using dictionary size of 5000, C = 1.0E-1, Accuracy = 0.7293